In [28]:
import pandas as pd
import numpy as np
import os 
import shutil
import os
import pandas as pd
import numpy as np
import shutil
from shutil import copyfile
from tensorflow.keras.layers import (GlobalAveragePooling2D, Activation, MaxPooling2D, Add, Conv2D, MaxPool2D, Dense,
                                     Flatten, InputLayer, BatchNormalization, Input, Embedding, Permute,
                                     Dropout, RandomFlip, RandomRotation, LayerNormalization, MultiHeadAttention,
                                     RandomContrast, Rescaling, Resizing, Reshape)
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer
import tensorflow as tf
import os
import pandas as pd
import numpy as np
import shutil
from shutil import copyfile
from tensorflow.keras.layers import (GlobalAveragePooling2D, Activation, MaxPooling2D, Add, Conv2D, MaxPool2D, Dense,
                                     Flatten, InputLayer, BatchNormalization, Input, Embedding, Permute,
                                     Dropout, RandomFlip, RandomRotation, LayerNormalization, MultiHeadAttention,
                                     RandomContrast, Rescaling, Resizing, Reshape)
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer
import tensorflow as tf

import pandas as pd
import numpy as np
import os 
import shutil
import tensorflow as tf
# Different layers
from tensorflow.keras.layers import MultiHeadAttention, Input, Dense, Reshape
from tensorflow.keras.layers import LayerNormalization, Layer
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers import Conv2D, Dropout
# For miscellaneous functions
from tensorflow.keras.datasets import fashion_mnist
from tensorflow import reduce_mean, float32, range, reshape
from tensorflow.keras import utils
from tensorflow.keras.metrics import TopKCategoricalAccuracy
from tensorflow.nn import gelu
from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import RandomRotation, RandomFlip, RandomContrast





# For tokenization
from tensorflow.image import extract_patches
import pandas as pd
# For math/arrays
import numpy as np
# For plotting
import matplotlib.pyplot as plt
# For profiling
import time
import os
import shutil
from shutil import copyfile

# vit


In [30]:
#config

CONFIGURATION = {
    "BATCH_SIZE": 32,
    "IM_SIZE": 121,
    "LEARNING_RATE": 1e-3,
    "N_EPOCHS": 20,
    "DROPOUT_RATE": 0.0,
    "REGULARIZATION_RATE": 0.0,
    "N_FILTERS": 6,
    "KERNEL_SIZE": 3,
    "N_STRIDES": 1,
    "POOL_SIZE": 2,
    "N_DENSE_1": 1024,
    "N_DENSE_2": 128,
    "NUM_CLASSES": 2,
    "PATCH_SIZE": 11,
    "PROJ_DIM": 363,
    "CLASS_NAMES": ['Benign','Malignant'],
}


In [31]:

class PatchEncoder(Layer):
    def __init__(self, N_PATCHES, HIDDEN_SIZE):
        super(PatchEncoder, self).__init__(name='patch_encoder')
        self.linear_projection = Dense(HIDDEN_SIZE)
        self.positional_embedding = Embedding(N_PATCHES, HIDDEN_SIZE)
        self.N_PATCHES = N_PATCHES

    def call(self, x):
        # Resize input images to (256, 256, 3)
        x = tf.image.resize(x, [121, 121])
        #x = tf.image.resize(x, [CONFIGURATION["IM_SIZE"], CONFIGURATION["IM_SIZE"]])
        
        patches = tf.image.extract_patches(
            images=x,
            sizes=[1, CONFIGURATION["PATCH_SIZE"], CONFIGURATION["PATCH_SIZE"], 1],
            strides=[1, CONFIGURATION["PATCH_SIZE"], CONFIGURATION["PATCH_SIZE"], 1],
            rates=[1, 1, 1, 1],
            padding='VALID')

        patch_dim = patches.shape[-1]
        patches = tf.reshape(patches, (tf.shape(x)[0], -1, patch_dim))

        embedding_input = tf.range(start=0, limit=self.N_PATCHES, delta=1)
        output = self.linear_projection(patches) + self.positional_embedding(embedding_input)

        return output

    
    
class TokenLearningLayer(Layer):
    def __init__(self, n_maps, token_dims):
        super(TokenLearningLayer, self).__init__(name='Token_Learning_Layer')
        self.n_maps = n_maps
        self.token_dims = token_dims
    def build(self, input_shape):
        self.input_embed_dim = input_shape[-1]
        self.model = Sequential()
        self.model.add(Reshape((self.token_dims[0],
        self.token_dims[1], self.input_embed_dim)))
        conv2D_layer = Conv2D(filters=self.n_maps, kernel_size=(3, 3),
                              activation=gelu, padding="same",
                              use_bias=False)
        self.model.add(conv2D_layer)
        conv2D_sigmoid_layer = Conv2D(filters=self.n_maps, kernel_size=(3, 3),
                                      activation="sigmoid", padding="same",
                                      use_bias=False)
        self.model.add(conv2D_sigmoid_layer)
        self.model.add(Reshape((-1, self.n_maps)))
        
    def call(self, inputs):
        total_tokens = inputs.shape[1]
        attn_wts = self.model(inputs)
        inputs = Reshape((total_tokens, 1, self.input_embed_dim))(inputs)
        attn_wts = Reshape((total_tokens, self.n_maps, 1))(attn_wts)
        attended_output = inputs*attn_wts
        output = reduce_mean(attended_output, axis=1)
        return output
    

class TransformerEncoder(Layer):
    def __init__(self, N_HEADS, HIDDEN_SIZE):
        super(TransformerEncoder, self).__init__(name='transformer_encoder')
        self.layer_norm_1 = LayerNormalization()
        self.layer_norm_2 = LayerNormalization()
        self.multi_head_att = MultiHeadAttention(num_heads=N_HEADS, key_dim=HIDDEN_SIZE)
        self.dense_1 = Dense(HIDDEN_SIZE, activation=tf.nn.gelu)
        self.dense_2 = Dense(HIDDEN_SIZE, activation=tf.nn.gelu)

    def call(self, inputs):
        x_1 = self.layer_norm_1(inputs)
        x_1 = self.multi_head_att(x_1, x_1)
        x_1 = Add()([x_1, inputs])
        x_2 = self.layer_norm_2(x_1)
        x_2 = self.dense_1(x_2)
        output = self.dense_2(x_2)
        output = Add()([output, x_1])
        return output

class ViT(Model):
    def __init__(self, N_HEADS, HIDDEN_SIZE, N_PATCHES, N_LAYERS, N_DENSE_UNITS):
        super(ViT, self).__init__(name='vision_transformer')
        self.N_LAYERS = N_LAYERS
        self.patch_encoder = PatchEncoder(N_PATCHES, HIDDEN_SIZE)
        self.trans_encoders_1 = [TransformerEncoder(N_HEADS, HIDDEN_SIZE) for _ in range(N_LAYERS)]
        self.trans_encoders_2 = [TransformerEncoder(N_HEADS, HIDDEN_SIZE) for _ in range(N_LAYERS)]

        self.dense_1 = Dense(N_DENSE_UNITS, activation=tf.nn.gelu)
        self.dense_2 = Dense(N_DENSE_UNITS, activation=tf.nn.gelu)
        self.TokenLearner = TokenLearningLayer(n_maps = 4,token_dims= (CONFIGURATION["PATCH_SIZE"] , CONFIGURATION["PATCH_SIZE"] , ) )

        self.dense_3 = Dense(CONFIGURATION["NUM_CLASSES"], activation='softmax')

    def call(self, inputs, training=None):
        x = self.patch_encoder(inputs)
        for i in range(self.N_LAYERS):
            x = self.trans_encoders_1[i](x)
        x = self.TokenLearner(x)
        for i in range(self.N_LAYERS):
            x = self.trans_encoders_2[i](x)
        
        
        
        x = Flatten()(x)
        
        
        x = self.dense_1(x)
        x = self.dense_2(x)
        return self.dense_3(x)

In [32]:
vit = ViT(
    N_HEADS = 4, HIDDEN_SIZE = 363, N_PATCHES = 121,
    N_LAYERS = 2, N_DENSE_UNITS = 128)


In [33]:
vit(tf.zeros([32,121,121,3]))

<tf.Tensor: shape=(32, 2), dtype=float32, numpy=
array([[0.6208165 , 0.37918344],
       [0.6208165 , 0.37918344],
       [0.6208165 , 0.37918344],
       [0.6208165 , 0.37918344],
       [0.6208165 , 0.37918344],
       [0.6208165 , 0.37918344],
       [0.6208165 , 0.37918344],
       [0.6208165 , 0.37918344],
       [0.6208165 , 0.37918344],
       [0.6208165 , 0.37918344],
       [0.6208165 , 0.37918344],
       [0.6208165 , 0.37918344],
       [0.6208165 , 0.37918344],
       [0.6208165 , 0.37918344],
       [0.6208165 , 0.37918344],
       [0.6208165 , 0.37918344],
       [0.6208165 , 0.37918344],
       [0.6208165 , 0.37918344],
       [0.6208165 , 0.37918344],
       [0.6208165 , 0.37918344],
       [0.6208165 , 0.37918344],
       [0.6208165 , 0.37918344],
       [0.6208165 , 0.37918344],
       [0.6208165 , 0.37918344],
       [0.6208165 , 0.37918344],
       [0.6208165 , 0.37918344],
       [0.6208165 , 0.37918344],
       [0.6208165 , 0.37918344],
       [0.6208165 , 0.37918

In [34]:
vit.summary()

Model: "vision_transformer"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ patch_encoder (PatchEncoder)    │ ?                      │       176,055 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_encoder             │ ?                      │     2,378,739 │
│ (TransformerEncoder)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_encoder             │ ?                      │     2,378,739 │
│ (TransformerEncoder)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_encoder             │ ?                      │     2,378,739 │
│ (TransformerEncoder)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_encoder             │ ?                      │     2,378,739 │
│ (TransformerEncoder)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_33 (Dense)                │ (32, 128)              │       185,984 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_34 (Dense)                │ (32, 128)              │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Token_Learning_Layer            │ ?                      │        13,212 │
│ (TokenLearningLayer)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_35 (Dense)                │ (32, 2)                │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,906,977 (37.79 MB)

 Trainable params: 9,906,977 (37.79 MB)

 Non-trainable params: 0 (0.00 B)

In [35]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy,CategoricalCrossentropy, SparseCategoricalCrossentropy
from tensorflow.keras.metrics import Accuracy,TopKCategoricalAccuracy, CategoricalAccuracy, SparseCategoricalAccuracy
from tensorflow.keras.metrics import CategoricalAccuracy, TopKCategoricalAccuracy, Precision, Recall


loss_function = CategoricalCrossentropy()

metrics = [
    CategoricalAccuracy(name="accuracy"),
    TopKCategoricalAccuracy(k=2, name="top_k_accuracy"),
    Precision(name="precision"),
    Recall(name="recall")
]

In [36]:
vit.compile(optimizer = Adam(learning_rate = CONFIGURATION["LEARNING_RATE"] ),
           loss = loss_function,
           metrics = metrics)

In [ ]:
train_directory = "/kaggle/input/for-isic-2024-paper/val_test_train_dataset/test_train_dataset/train"
test_directory = "/kaggle/input/for-isic-2024-paper/val_test_train_dataset/test_train_dataset/test"

val_directory = "/kaggle/input/for-isic-2024-paper/val_test_train_dataset/test_train_dataset/val"


train_dataset = tf.keras.utils.image_dataset_from_directory(
    train_directory,
    labels='inferred',
    label_mode='categorical',
    class_names=CONFIGURATION["CLASS_NAMES"],
    color_mode='rgb',
    batch_size=CONFIGURATION["BATCH_SIZE"],
    image_size=(CONFIGURATION["IM_SIZE"], CONFIGURATION["IM_SIZE"]),
    shuffle=True,
    seed=99,
)
test_dataset = tf.keras.utils.image_dataset_from_directory(
    test_directory,
    labels='inferred',
    label_mode='categorical',
    class_names=CONFIGURATION["CLASS_NAMES"],
    color_mode='rgb',
    batch_size=1,#CONFIGURATION["BATCH_SIZE"],
    image_size=(CONFIGURATION["IM_SIZE"], CONFIGURATION["IM_SIZE"]),
    shuffle=True,
    seed=99,
)


val_dataset = tf.keras.utils.image_dataset_from_directory(
    val_directory,
    labels='inferred',
    label_mode='categorical',
    class_names=CONFIGURATION["CLASS_NAMES"],
    color_mode='rgb',
    batch_size=1,#CONFIGURATION["BATCH_SIZE"],
    image_size=(CONFIGURATION["IM_SIZE"], CONFIGURATION["IM_SIZE"]),
    shuffle=True,
    seed=99,
)

In [ ]:
augment_layers = tf.keras.Sequential([
  RandomRotation(factor = (-0.025, 0.025)),
  RandomFlip(mode='horizontal',),
  RandomContrast(factor=0.1),
])
def augment_layer(image, label):
    return augment_layers(image, training = True), label

In [ ]:
training_dataset = (
    train_dataset
    .map(augment_layer, num_parallel_calls = tf.data.AUTOTUNE)
    .prefetch(tf.data.AUTOTUNE)
)
testing_dataset = (
    test_dataset
    .prefetch(tf.data.AUTOTUNE)
)
validation_dataset = (
    val_dataset
    .prefetch(tf.data.AUTOTUNE)
)

In [ ]:
training_dataset

In [ ]:
import time

start_time = time.time()

history = vit.fit(
    training_dataset,
    validation_data = validation_dataset,
    epochs = CONFIGURATION["N_EPOCHS"],
verbose = 1)

end_time = time.time()

# Calculate the time taken
time_taken = end_time - start_time

# Print the time taken in a readable format
print(f"Time taken for training: {time_taken:.2f} seconds")


In [ ]:
trainin time = time_taken

In [ ]:
import matplotlib.pyplot as plt


plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Accuracy vs. Validation Accuracy')
plt.legend()

# Plot loss vs val_loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss vs. Validation Loss')
plt.legend()

plt.show()


In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, f1_score

predictions = model.predict(test_dataset)
predicted_classes = np.argmax(predictions, axis=1)

true_labels = np.concatenate([y for x, y in test_dataset], axis=0)
true_classes = np.argmax(true_labels, axis=1)

conf_matrix = confusion_matrix(true_classes, predicted_classes)
print("Confusion Matrix:\n", conf_matrix)

f1 = f1_score(true_classes, predicted_classes, average='weighted')
print("F1 Score: {:.4f}".format(f1))


In [ ]:
evaluation_results = model.evaluate(testing_dataset)
for metric_name, metric_value in zip(model.metrics_names, evaluation_results):
    print(f"{metric_name}: {metric_value:.4f}")